In [99]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import math
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# 경고 메시지 숨기기
warnings.filterwarnings("ignore")

In [100]:
df = pd.read_csv("./data/data_total.csv")

### 날짜 타입 변경 및 년/월/일 분리

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856566 entries, 0 to 856565
Data columns (total 26 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              856566 non-null  int64  
 1   msno                    856566 non-null  int64  
 2   city                    856566 non-null  int64  
 3   bd                      856566 non-null  int64  
 4   gender                  368210 non-null  object 
 5   registered_via          856566 non-null  int64  
 6   registration_init_time  856566 non-null  object 
 7   is_churn                856566 non-null  int64  
 8   is_back                 856566 non-null  int64  
 9   payment_plan_sum        856566 non-null  int64  
 10  plan_list_price         856566 non-null  int64  
 11  actual_amount_paid      856566 non-null  int64  
 12  discount_rate           856566 non-null  float64
 13  is_auto_renew           856566 non-null  float64
 14  membership_expire_da

In [102]:
def int_to_date(df):
    # membership_expire_date와 registration_init_time은 'YYYY-MM-DD' 형식으로 되어 있으므로, 그에 맞춰 변환
    df['membership_expire_date'] = pd.to_datetime(df['membership_expire_date'], errors='coerce')
    df['registration_init_time'] = pd.to_datetime(df['registration_init_time'], errors='coerce')
    
    # log_start와 log_end는 'YYYYMMDD' 형식이므로, 그에 맞춰 변환
    df['log_start'] = pd.to_datetime(df['log_start'].astype(str), format='%Y%m%d', errors='coerce')
    df['log_end'] = pd.to_datetime(df['log_end'].astype(str), format='%Y%m%d', errors='coerce')

    return df

# 함수 실행
df = int_to_date(df)

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856566 entries, 0 to 856565
Data columns (total 26 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Unnamed: 0              856566 non-null  int64         
 1   msno                    856566 non-null  int64         
 2   city                    856566 non-null  int64         
 3   bd                      856566 non-null  int64         
 4   gender                  368210 non-null  object        
 5   registered_via          856566 non-null  int64         
 6   registration_init_time  856566 non-null  datetime64[ns]
 7   is_churn                856566 non-null  int64         
 8   is_back                 856566 non-null  int64         
 9   payment_plan_sum        856566 non-null  int64         
 10  plan_list_price         856566 non-null  int64         
 11  actual_amount_paid      856566 non-null  int64         
 12  discount_rate           856566

* 년,월,일 분리

In [104]:

def split_date_columns(df, column_name):
    # 컬럼을 datetime 형식으로 변환 (변환 실패시 NaT로 처리)
    df[column_name] = pd.to_datetime(df[column_name], errors='coerce')
    
    # 연도, 월, 일 컬럼 추가
    df[f'{column_name}_year'] = df[column_name].dt.year
    df[f'{column_name}_month'] = df[column_name].dt.month
    df[f'{column_name}_day'] = df[column_name].dt.day
    
    # 원본 컬럼 삭제
    df = df.drop(columns=[column_name])
    
    return df

In [105]:
df = split_date_columns(df, 'registration_init_time')
df = split_date_columns(df, 'membership_expire_date')
df = split_date_columns(df, 'log_end')
df = split_date_columns(df, 'log_start')

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856566 entries, 0 to 856565
Data columns (total 34 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Unnamed: 0                    856566 non-null  int64  
 1   msno                          856566 non-null  int64  
 2   city                          856566 non-null  int64  
 3   bd                            856566 non-null  int64  
 4   gender                        368210 non-null  object 
 5   registered_via                856566 non-null  int64  
 6   is_churn                      856566 non-null  int64  
 7   is_back                       856566 non-null  int64  
 8   payment_plan_sum              856566 non-null  int64  
 9   plan_list_price               856566 non-null  int64  
 10  actual_amount_paid            856566 non-null  int64  
 11  discount_rate                 856566 non-null  float64
 12  is_auto_renew                 856566 non-nul

### 성별 NAN값 제거 후 인코딩

In [107]:
df_filtered = df[df['gender'].notna()]

In [ ]:
def encode_gender(df):
    # 성별 인코딩: M -> 1, F -> 0, nan -> -1 (또는 원하는 값으로 변경)
    df['gender_encoded'] = df['gender'].map({'male': 1, 'female': 0})
    df = df.drop(columns=["gender"])
    
    return df

In [ ]:
# df['gender'].isnull().sum()

0

In [ ]:
# df = encode_gender(df_filtered)

In [110]:
df["gender"]

0           male
1           male
2           male
4         female
5         female
           ...  
856540      male
856547      male
856548    female
856559      male
856564    female
Name: gender, Length: 368210, dtype: object

### 나이 전처리

In [ ]:
# df["bd"] = df["bd"].abs()

In [ ]:
# # 0인 값이 많아서 Age 중앙값이 0으로 나옴. 10보다 큰 Age 값의 중앙값

# median_df_bd = df[df["bd"] > 11]["bd"].median()
# median_df_bd

28.0

In [ ]:
# df["bd"] = df["bd"].apply(lambda x: x if (x != 0) and (x < 11 or x > 100) else np.nan)

In [ ]:
# df_filtered = df[df['bd'].notna()]

In [ ]:
# df['bd'].unique()

array([      nan, 1.090e+02, 1.170e+02, 1.050e+02, 1.000e+00, 1.060e+02,
       5.000e+00, 2.200e+02, 1.020e+02, 4.000e+00, 1.030e+02, 1.037e+03,
       1.370e+02, 3.000e+00, 1.032e+03, 1.110e+02, 1.070e+02, 1.160e+02,
       1.000e+01, 9.460e+02, 8.000e+00, 1.150e+02, 6.000e+00, 1.040e+02,
       9.360e+02, 1.100e+02, 1.043e+03, 7.000e+00, 9.510e+02, 1.260e+02,
       9.300e+02, 1.034e+03, 1.026e+03, 1.120e+02, 1.053e+03, 7.780e+02,
       9.000e+00, 9.500e+02, 1.017e+03, 1.030e+03, 1.036e+03, 1.040e+03,
       1.530e+02, 1.048e+03, 7.430e+02, 1.031e+03, 1.190e+02, 1.051e+03,
       1.049e+03, 1.380e+02, 2.000e+00, 1.029e+03, 9.480e+02, 1.440e+02,
       1.010e+02, 1.052e+03, 1.310e+02, 8.230e+02, 3.190e+02, 9.390e+02,
       9.470e+02, 9.410e+02, 3.400e+02, 9.310e+02, 1.033e+03, 9.330e+02,
       1.022e+03, 1.220e+02, 1.600e+02, 5.190e+02, 1.390e+02, 1.009e+03,
       1.970e+02, 9.640e+02, 1.042e+03, 1.130e+02, 8.370e+02, 3.200e+02,
       7.860e+02, 1.035e+03, 8.210e+02, 1.970e+03, 

### 정규화

In [116]:
# X_train, X_test, y_train, y_test = train_test_split(train_data.drop('gender',axis=1), train_data['gender'], test_size=0.3, random_state=42)

# std_scaler = StandardScaler()
# train_scaled = std_scaler.fit_transform(X_train)
# test_scaled = std_scaler.transform(X_test)

### 데이터 저장

In [117]:
df_filtered.to_csv("./data/data_total_split_datetime.csv")